# Introduction to JuliaDiffEq Project

## Presentation based on official "Basic Introduction Via Ordinary Differential Equations" [An Intro to DifferentialEquations.jl](https://github.com/JuliaDiffEq/DiffEqTutorials.jl/blob/master/notebook/introduction/01-ode_introduction.ipynb).

## _JuliaDiffEq is a Github organization created to unify the packages for solving differential equations in Julia. By providing a diverse set of tools with a common interface, we provide a modular, easily-extendable, and highly performant ecosystem for solving various forms of differential equations._

From the homepage of [JuliaDiffEq](https://juliadiffeq.org/)

## Lead person of these project is [Chris Rackauckas](http://chrisrackauckas.com/), Applied Mathematics Instructor at the Massachusetts Institute of Technology and Senior Research Analyst at University of Maryland, Baltimore, School of Pharmacy.

## Curently he is studing scientific AI, such a neural differential equations.

In [ ]:
# import Pkg; Pkg.add(["DifferentialEquations", "Plots", "Latexify"])

In [ ]:
using DifferentialEquations
using Plots; gr()

## Simplest differential equations

\begin{equation*}
    u'( t ) = f( u( t ), p, t )
\end{equation*}

$u( t )$ - solution of equation.

$p$ - parameters.

$t$ - time.

In [ ]:
f(u, p, t) = 0.98u
u0 = 1.0
tspan = (0.0, 1.0)

In [ ]:
prob = ODEProblem(f, u0, tspan)

In [ ]:
sol = solve(prob)

In [ ]:
plot(sol, linewidth=5, title="Solution to the linear ODE with a thick line",
    xaxis="Time (t)", yaxis="u(t) (in μm)", label="My Thick Line!") # legend = false

In [ ]:
plot!(sol.t, t -> 1.0*exp(0.98t), lw=3, ls=:dash, label="True Solution!")

In [ ]:
sol

In [ ]:
typeof(sol)

In [ ]:
sol.t

In [ ]:
sol.u

In [ ]:
sol(0.45)

In [ ]:
exp(0.98 * 0.45)

In [ ]:
sol = solve(prob, abstol=1e-8, reltol=1e-8)

In [ ]:
plot(sol, ylabel="u( t )")
plot!(sol.t, t -> 1.0*exp(0.98t), lw=3, ls=:dash, label="True Solution!")

In [ ]:
sol = solve(prob, alg_hints=[:stiff])

In [ ]:
sol(0.45)

In [ ]:
exp(0.98*0.45)

In [ ]:
sol = solve(prob, Tsit5(), reltol=1e-6)

In [ ]:
A = Array(sol)

## Lotka-Volterra equations

\begin{align}
    \frac{ d x( t ) }{ dt } &= a x( t ) - b x( t ) y( t ) \\
    \frac{ d y( t ) }{ dt } &= -c y( t ) + d x( t ) y( t )
\end{align}

$x( t )$ - number of rabbits.

$y( t )$ - number of foxes.

$a$, $b$, $c$, $d$ - parameters of interaction between rabits and foxes.

In [ ]:
function lotka_volterra!(du, u, p, t)
    du[1] = p[1]*u[1] - p[2]*u[1]*u[2]
    du[2] = -p[3]*u[2] + p[4]*u[1]*u[2]
end

In [ ]:
u0 = [1.0, 1.0]
p = (1.5, 1.0, 3.0, 1.0)
tspan = (0.0, 15.0)
prob = ODEProblem(lotka_volterra!, u0, tspan, p)
sol = solve(prob)

In [ ]:
plot(sol, xlabel="t", ylabel="number of animals (in thousands)", title="Rabbits and foxes population in Lotka-Volttera model")

In [ ]:
phasespace_plot = plot(sol, vars=(1,2), xlabel="rabbits (in thousands)", ylabel="foxes (in thousands)")

In [ ]:
savefig(phasespace_plot, "phasespace_plot.png")

### This is not easy to read

In [ ]:
function lotka_volterra!(du, u, p, t)
    du[1] = p[1]*u[1] - p[2]*u[1]*u[2]
    du[2] = -p[3]*u[2] + p[4]*u[1]*u[2]
end

## Some Lisp-like macro magic

In [ ]:
# import Pkg; Pkg.add("ParameterizedFunctions")
using ParameterizedFunctions

In [ ]:
lv! = @ode_def LotkaVolterra begin
    dx = a*x - b*x*y
    dy = -c*y + d*x*y
end a b c d

In [ ]:
u0 = [1.0,1.0]
p = (1.5,1.0,3.0,1.0)
tspan = (0.0,15.0)
prob = ODEProblem(lv!, u0, tspan, p)
sol = solve(prob)
plot(sol, xlabel="t", ylabel="number of animals (in thousands)", title="Rabbits and foxes population in Lotka-Volttera model")

In [ ]:
plot(sol, vars=(1,2), xlabel="rabbits (in thousands)", ylabel="foxes (in thousands)")

In [ ]:
lv!.Jex

In [ ]:
using Latexify

latexify(lv!)

## Matrix valued problem

In [ ]:
 import Pkg; Pkg.add("StaticArrays")
using StaticArrays

In [ ]:
A = @SMatrix [ 1.0  0.0 0.0 -5.0
               4.0 -2.0 4.0 -3.0
              -4.0  0.0 0.0  1.0
               5.0 -2.0 2.0  3.0]
u0 = @SMatrix [1.0 0.0
               0.0 1.0
               0.0 0.0
               1.0 0.0]
tspan = (0.0,1.0)
f(u, p, t) = A*u
prob = ODEProblem(f, u0, tspan)
sol = solve(prob)

In [ ]:
sol[3]

## [Hénon–Heiles system](https://en.wikipedia.org/wiki/H%C3%A9non%E2%80%93Heiles_system)

\begin{equation*}
    H( x, y, p_{ x }, p_{ y } ) = \frac{ 1 }{ 2 } ( p_{ x }^{ 2 } + p_{ y }^{ 2 } )
    + \frac{ 1 }{ 2 } ( x^{ 2 } + y^{ 2 } ) + \lambda \left( x^{ 2 } y - \frac{ y^{ 2 } }{ 3 } \right)
\end{equation*}

\begin{align}
    \frac{dx( t )}{dt} &= p_{ x }( t ) \\[0.4em]
    \frac{dp_{ x }( t )}{dt} &=  - x - 2 \lambda x( t ) y( t ) \\[0.4em]
    \frac{dy( t )}{dt} &= p_{ y }( t ) \\[0.4em]
    \frac{dp_{ y }( t )}{dt} &=  - y( t ) - \lambda \left( x( t )^{2} - y( t )^{2} \right)
\end{align}

![Henon-Heiles system](../SeminarPictures/Henon_Heiles_potential.svg)

In [ ]:
# import Pkg; Pkg.add("ParameterizedFunctions")
using DifferentialEquations, ParameterizedFunctions

In [ ]:
HH_sys! = @ode_def HenonHeiles begin
    dx = px
    dpx = -x - 2λ * x * y
    dy = py
    dpy = -y - λ * (x^2 - y^2)
end λ

In [ ]:
latexify(HH_sys!)

In [ ]:
latexify(HH_sys!) |> print

In [ ]:
typeof(HH_sys!)

In [ ]:
# λ = 1
# init0 = [1.0, 0.0, 0.0, 0.0]
# prob = ODEProblem(HH_sys!, init0, (0.0, 1.0), λ)
# sol = solve(prob)

In [ ]:
# plot(sol, vars=(1, 3), xlabel="x", ylabel="y", xlim=(-3,3), ylim=(-3,3), title="Henon-Helies system")

In [ ]:
λ = 1
init0 = [0.5, 0.0, 0.0, 0.0]
prob = ODEProblem(HH_sys!, init0, (0.0, 5.0), λ)
sol = solve(prob)

In [ ]:
plot(sol, vars=(1, 3),
    width=1,
    xlabel="x", ylabel="y", xlim=(-1,1), ylim=(-1,1), title="Henon-Helies system")

In [ ]:
typeof(sol)

In [ ]:
prob = ODEProblem(HH_sys!, init0, (0.0, 10.0), λ)
sol = solve(prob)
plot(sol, vars=(1, 3),
    width=1,
    xlabel="x", ylabel="y", xlim=(-1,1), ylim=(-1,1), title="Henon-Helies system")

In [ ]:
prob = ODEProblem(HH_sys!, init0, (0.0, 20.0), λ)
sol = solve(prob)
plot(sol, vars=(1, 3),
    width=1,
    xlabel="x", ylabel="y", xlim=(-1,1), ylim=(-1,1), title="Henon-Helies system")

In [ ]:
prob = ODEProblem(HH_sys!, init0, (0.0, 50.0), λ)
sol = solve(prob)
plot(sol, vars=(1, 3),
    width=1,
    xlabel="x", ylabel="y", xlim=(-1,1), ylim=(-1,1), title="Henon-Helies system")

In [ ]:
# init0 = [0.0, 0.5, 0.1, 0.0]
prob = ODEProblem(HH_sys!, init0, (0.0, 100.0), λ)
sol = solve(prob)
plot(sol, vars=(1, 3),
    width=1,
    xlabel="x", ylabel="y", xlim=(-1,1), ylim=(-1,1), title="Henon-Helies system")

In [ ]:
prob = ODEProblem(HH_sys!, init0, (0.0, 300.0), λ)
sol = solve(prob)
plot(sol, vars=(1, 3),
    width=1,
    xlabel="x", ylabel="y", xlim=(-1,1), ylim=(-1,1), title="Henon-Helies system")

In [ ]:
prob = ODEProblem(HH_sys!, init0, (0.0, 1000.0), λ)
sol = solve(prob)
plot(sol, vars=(1, 3),
    width=1,
    xlabel="x", ylabel="y", xlim=(-1,1), ylim=(-1,1), title="Henon-Helies system")

In [ ]:
prob = ODEProblem(HH_sys!, init0, (0.0, 2000.0), λ)
sol = solve(prob)
plot(sol, vars=(1, 3),
    width=1,
    xlabel="x", ylabel="y", xlim=(-1,1), ylim=(-1,1), title="Henon-Helies system")

In [ ]:
plot(sol, xlabel="t", ylabel="postions and momentums", title="Henon-Helies system")

In [ ]:
prob = ODEProblem(HH_sys!, init0, (0.0, 3000.0), λ)
sol = solve(prob)
plot(sol, vars=(1, 3),
    width=1,
    xlabel="x", ylabel="y", xlim=(-1,1), ylim=(-1,1), title="Henon-Helies system")

In [ ]:
plot(sol, xlabel="t", ylabel="postions and momentums", title="Henon-Helies system")

In [ ]:
λ = 0
init0 = [1.9, 0, 0, 0]
prob = ODEProblem(HH_sys!, init0, (0, 10.0), λ)
sol = solve(prob)

In [ ]:
plot(sol, vars=(1,2), xlabel="x", ylabel="p_x", title="Henon-Helies system")